In [ ]:
import sys,json
!{sys.executable} -m pip install kfp==1.4.0 kfp-server-api==1.2.0 --user >/dev/null

### MM DETAILS

In [ ]:
with open("details.txt","r") as f:
    mm_details = json.load(f)

mm_name = mm_details['MONITOR_NAME']
data_source = mm_details['DATA_SOURCE']
input_train_type = mm_details['INPUT_TRAIN_TYPE']
OUTSIDE_DKUBE = mm_details['OUTSIDE_DKUBE']

## Importing libraries

In [ ]:
import json
import kfp
import kfp.dsl as dsl
import kfp.compiler as compiler
from kfp import components
from kubernetes import client as k8s_client

import os
import json
import kfp
import string
import random
import kfp.dsl as dsl
import kfp.compiler as compiler
from kfp import components
from kubernetes import client as k8s_client

from dkube.sdk.api import DkubeApi
from dkube.sdk.rsrcs import DkubeCode
from dkube.sdk.rsrcs import DkubeDataset
from dkube.sdk.rsrcs import DkubeModel

## Loading Components

In [ ]:
dkube_preprocessing_op = components.load_component_from_file("/mnt/dkube/pipeline/components/preprocess/component.yaml")
dkube_training_op = components.load_component_from_file("/mnt/dkube/pipeline/components/training/component.yaml")
dkube_serving_op  = components.load_component_from_file("/mnt/dkube/pipeline/components/serving/component.yaml")

In [ ]:
if not OUTSIDE_DKUBE:
    token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
    username =  os.getenv("USER")
    url = os.getenv("DKUBE_URL")

else:
    print("Taking information from details.txt file")
    token = mm_details['TOKEN']
    username = mm_details['DKUBEUSERNAME']
    url = mm_details['URL']

api = DkubeApi(URL=url,token=token)
client = kfp.Client(host=os.getenv("KF_PIPELINES_ENDPOINT"), existing_token=token, namespace=os.getenv("USER"))

## Pipeline Inputs

In [ ]:
training_program = 'insurance'
if (data_source == 'local' or data_source == 'aws_s3') and input_train_type == 'retraining':
    input_training_dataset = mm_name+'-groundtruth'
elif data_source == 'sql':
    input_training_dataset = 'insurance-sql-data'
else:
    input_training_dataset = 'insurance-data'


## Preprocessing stage inputs
preprocessing_script =f"pip3 install pymysql --user;python insurance/preprocessing.py --data_source {data_source} --train_type {input_train_type} --monitor_name {mm_name} --user {username}"
input_dataset_mount = ['/data']
output_dataset = 'insurance-training-data'
output_mount_path = ['/train-data']

## Training stage inputs
training_script = "python insurance/training.py"
model_name = 'insurance-model'
output_model_mount = "/model"

## Pipeline Definition

In [ ]:
@kfp.dsl.pipeline(
    name='training-pipeline',
    description='insurance-training-pl'
)
def insurance_pipeline(token):
    
    preprocessing = dkube_preprocessing_op(
                                    auth_token=str(token),
                                    container=json.dumps({"image": "ocdr/d3-datascience-sklearn:v0.23.2-1"}),
                                    program=str(training_program),
                                    datasets = json.dumps([str(input_training_dataset)]),
                                    input_dataset_mounts = json.dumps(input_dataset_mount),
                                    run_script=str(preprocessing_script),
                                    outputs=json.dumps([str(output_dataset)]),
                                    output_mounts=json.dumps(output_mount_path)).set_display_name("data-generation")
    
    train       = dkube_training_op(container=json.dumps({"image": "docker.io/ocdr/d3-datascience-sklearn:v0.23.2"}),
                                    framework="sklearn", version="0.23.2",
                                    program=str(training_program), 
                                    run_script=str(training_script),
                                    datasets=json.dumps([str(output_dataset)]), outputs=json.dumps([str(model_name)]),
                                    input_dataset_mounts=json.dumps(output_mount_path),
                                    output_mounts=json.dumps([str(output_model_mount)]),
                                    auth_token=token).after(preprocessing)
    
    serving     = dkube_serving_op(model=train.outputs['artifact'], device='cpu', 
                                    serving_image=json.dumps({"image": "ocdr/sklearnserver:0.23.2"}),
                                    transformer_image =json.dumps({"image": "docker.io/ocdr/d3-datascience-sklearn:v0.23.2"}),
                                    transformer_project=str(training_program),
                                    transformer_code='insurance/transformer.py', auth_token=token).after(train)

    

In [ ]:
client.create_run_from_pipeline_func(insurance_pipeline, arguments={'token':token})